In [ ]:
"""
DeepConvNet Baseline - Motor Imagery Classification
====================================================

Instalar dependencias:
    pip install "numpy>=1.24.0,<2.0.0" mne scikit-learn torch tqdm

Resultado esperado: ~65% accuracy (mejor modelo del estudio)
"""

In [1]:
import os
import re
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
from glob import glob
from tqdm import tqdm

In [3]:
import mne
from mne.io import read_epochs_eeglab

In [4]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [6]:
# Definición del modelo DeepConvNet
# Basado en Schirrmeister et al. (2017)

class DeepConvNet(nn.Module):
    """
    DeepConvNet para clasificación de EEG
    Arquitectura específicamente diseñada para señales EEG
    """
    
    def __init__(self, n_channels=64, n_classes=2, n_timepoints=1152, dropout=0.5):
        super(DeepConvNet, self).__init__()
        
        # Bloque 1: Convolución temporal
        self.conv1 = nn.Conv2d(1, 25, kernel_size=(1, 10), stride=1)
        
        # Bloque 2: Convolución espacial
        self.conv2 = nn.Conv2d(25, 25, kernel_size=(n_channels, 1), stride=1)
        self.bn1 = nn.BatchNorm2d(25)
        self.pool1 = nn.MaxPool2d(kernel_size=(1, 3), stride=(1, 3))
        
        # Bloque 3
        self.conv3 = nn.Conv2d(25, 50, kernel_size=(1, 10), stride=1)
        self.bn2 = nn.BatchNorm2d(50)
        self.pool2 = nn.MaxPool2d(kernel_size=(1, 3), stride=(1, 3))
        
        # Bloque 4
        self.conv4 = nn.Conv2d(50, 100, kernel_size=(1, 10), stride=1)
        self.bn3 = nn.BatchNorm2d(100)
        self.pool3 = nn.MaxPool2d(kernel_size=(1, 3), stride=(1, 3))
        
        # Bloque 5
        self.conv5 = nn.Conv2d(100, 200, kernel_size=(1, 10), stride=1)
        self.bn4 = nn.BatchNorm2d(200)
        self.pool4 = nn.MaxPool2d(kernel_size=(1, 3), stride=(1, 3))
        
        self.dropout = nn.Dropout(dropout)
        
        # Capa FC se crea dinámicamente
        self.fc = None
        
    def forward(self, x):
        # x shape: (batch, 1, channels, timepoints)
        
        # Bloque 1-2
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.bn1(x)
        x = torch.relu(x)
        x = self.pool1(x)
        x = self.dropout(x)
        
        # Bloque 3
        x = self.conv3(x)
        x = self.bn2(x)
        x = torch.relu(x)
        x = self.pool2(x)
        x = self.dropout(x)
        
        # Bloque 4
        x = self.conv4(x)
        x = self.bn3(x)
        x = torch.relu(x)
        x = self.pool3(x)
        x = self.dropout(x)
        
        # Bloque 5
        x = self.conv5(x)
        x = self.bn4(x)
        x = torch.relu(x)
        x = self.pool4(x)
        x = self.dropout(x)
        
        # Flatten
        batch_size = x.size(0)
        x = x.view(batch_size, -1)
        
        # Crear FC dinámicamente si no existe
        if self.fc is None:
            fc_input_size = x.size(1)
            self.fc = nn.Linear(fc_input_size, 2).to(x.device)
        
        x = self.fc(x)
        return x

print("✅ Modelo DeepConvNet definido")

✅ Modelo DeepConvNet definido


In [7]:
print("="*70)
print("DEEPCONVNET BASELINE - MOTOR IMAGERY CLASSIFICATION")
print("="*70)
print(f"PyTorch version: {torch.__version__}")
print(f"MNE version: {mne.__version__}")
print(f"NumPy version: {np.__version__}")

DEEPCONVNET BASELINE - MOTOR IMAGERY CLASSIFICATION
PyTorch version: 2.2.2
MNE version: 1.10.2
NumPy version: 1.26.4


============================================================================
CONFIGURACIÓN
============================================================================

In [8]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [9]:
# Parámetros de preprocesamiento
LOW_FREQ = 8.0
HIGH_FREQ = 30.0

In [10]:
# Hiperparámetros
BATCH_SIZE = 16
LR = 1e-3
EPOCHS = 50
PATIENCE = 10
DROPOUT = 0.5

In [11]:
# Device
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\n🖥️  Usando dispositivo: {DEVICE}")


🖥️  Usando dispositivo: cpu


In [12]:
# Rutas
ROOT = Path.cwd()  # Cambiado para notebook
LEFT_DIR = ROOT / 'left_imag'
RIGHT_DIR = ROOT / 'right_imag'
OUT_DIR = ROOT / 'results' / 'deepconvnet'  # Simplificado
OUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"📁 Directorio de salida: {OUT_DIR}")

📁 Directorio de salida: /Users/manueljurado/Downloads/datos_BCI/results/deepconvnet


============================================================================
CARGA DE DATOS
============================================================================

In [14]:
def load_dataset():
    """
    Carga datos de imaginería motora desde archivos .set
    Returns: X (trials, channels, timepoints), y (labels), subjects (subject_ids)
    """
    X_list, y_list, subjects_list = [], [], []
    
    # Cargar Left
    left_files = sorted(glob(str(LEFT_DIR / '*.set')))
    for fpath in tqdm(left_files, desc='Left'):
        subj = re.search(r'S(\d+)', Path(fpath).name).group(1)
        epochs = read_epochs_eeglab(fpath, verbose=False)
        epochs.filter(LOW_FREQ, HIGH_FREQ, fir_design='firwin', verbose=False)
        epochs.resample(128, npad='auto', verbose=False)
        
        # epochs.get_data() returns (n_epochs, n_channels, n_times)
        # Each epoch is a separate trial
        data = epochs.get_data()  # (n_epochs, channels, timepoints)
        for trial in data:
            X_list.append(trial)
            y_list.append(0)  # Left = 0
            subjects_list.append(int(subj))
    
    # Cargar Right
    right_files = sorted(glob(str(RIGHT_DIR / '*.set')))
    for fpath in tqdm(right_files, desc='Right'):
        subj = re.search(r'S(\d+)', Path(fpath).name).group(1)
        epochs = read_epochs_eeglab(fpath, verbose=False)
        epochs.filter(LOW_FREQ, HIGH_FREQ, fir_design='firwin', verbose=False)
        epochs.resample(128, npad='auto', verbose=False)
        
        data = epochs.get_data()  # (n_epochs, channels, timepoints)
        for trial in data:
            X_list.append(trial)
            y_list.append(1)  # Right = 1
            subjects_list.append(int(subj))
    
    X = np.array(X_list)  # (trials, channels, timepoints)
    y = np.array(y_list)
    subjects = np.array(subjects_list)
    
    print(f"\n  Dataset: {X.shape}, Labels: {np.bincount(y)}, Sujetos: {len(np.unique(subjects))}")
    
    return X, y, subjects

In [15]:
print("\n" + "="*70)
print("CARGANDO DATOS")
print("="*70)
X, y, subjects = load_dataset()


CARGANDO DATOS


Right: 100%|██████████| 20/20 [00:09<00:00,  2.02it/s]



  Dataset: (880, 64, 1152), Labels: [442 438], Sujetos: 20


============================================================================
DATASET Y DATALOADER
============================================================================

In [17]:
class EEGDataset(Dataset):
    def __init__(self, X, y):
        # Normalización per-channel z-score (across all trials)
        self.X = X.copy()
        for ch in range(self.X.shape[1]):
            ch_data = self.X[:, ch, :].flatten()
            mean = ch_data.mean()
            std = ch_data.std()
            self.X[:, ch, :] = (self.X[:, ch, :] - mean) / (std + 1e-8)
        
        self.y = y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        trial = self.X[idx]  # (channels, timepoints)
        label = self.y[idx]
        
        # Add channel dimension: (1, channels, timepoints)
        trial_tensor = torch.FloatTensor(trial).unsqueeze(0)
        label_tensor = torch.LongTensor([label])[0]
        
        return trial_tensor, label_tensor

In [18]:
# Split train/test (80-20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_SEED, stratify=y
)

In [19]:
print(f"\nTrain: {len(X_train)}, Test: {len(X_test)}")
print(f"Train labels: {np.bincount(y_train)}")
print(f"Test labels: {np.bincount(y_test)}")


Train: 704, Test: 176
Train labels: [354 350]
Test labels: [88 88]


In [20]:
# Crear datasets
train_dataset = EEGDataset(X_train, y_train)
test_dataset = EEGDataset(X_test, y_test)

In [21]:
# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

============================================================================
MODELO
============================================================================

In [22]:
n_channels = X_train.shape[1]
n_timepoints = X_train.shape[2]
n_classes = len(np.unique(y))

In [23]:
print("\n" + "="*70)
print("INICIALIZANDO MODELO")
print("="*70)
print(f"  Channels: {n_channels}")
print(f"  Timepoints: {n_timepoints}")
print(f"  Classes: {n_classes}")
print(f"  Dropout: {DROPOUT}")


INICIALIZANDO MODELO
  Channels: 64
  Timepoints: 1152
  Classes: 2
  Dropout: 0.5


In [24]:
model = DeepConvNet(n_channels, n_classes, n_timepoints, dropout=DROPOUT).to(DEVICE)

In [25]:
# Contar parámetros
n_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\n✅ Modelo creado con {n_params:,} parámetros")


✅ Modelo creado con 303,900 parámetros


============================================================================
TRAINING
============================================================================

In [26]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=1e-4)

In [27]:
# Early stopping
best_val_loss = float('inf')
patience_counter = 0
best_model_state = None

In [28]:
print("\n" + "="*70)
print("INICIANDO ENTRENAMIENTO")
print("="*70)


INICIANDO ENTRENAMIENTO


In [29]:
history = {'train_loss': [], 'train_acc': []}

In [30]:
for epoch in range(EPOCHS):
    # Training
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(DEVICE), batch_y.to(DEVICE)
        
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * batch_X.size(0)
        _, predicted = torch.max(outputs, 1)
        train_total += batch_y.size(0)
        train_correct += (predicted == batch_y).sum().item()
    
    train_loss /= train_total
    train_acc = train_correct / train_total
    
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    
    # Early stopping
    if train_loss < best_val_loss:
        best_val_loss = train_loss
        patience_counter = 0
        best_model_state = model.state_dict().copy()
    else:
        patience_counter += 1
    
    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1}/{EPOCHS} | Loss: {train_loss:.4f} | Acc: {train_acc:.4f} | Patience: {patience_counter}/{PATIENCE}")
    
    if patience_counter >= PATIENCE:
        print(f"\n⚠️  Early stopping en epoch {epoch+1}")
        break

Epoch 5/50 | Loss: 0.7733 | Acc: 0.5156 | Patience: 1/10
Epoch 10/50 | Loss: 0.7304 | Acc: 0.5369 | Patience: 2/10
Epoch 15/50 | Loss: 0.7186 | Acc: 0.5213 | Patience: 3/10
Epoch 20/50 | Loss: 0.7242 | Acc: 0.5625 | Patience: 3/10
Epoch 25/50 | Loss: 0.6870 | Acc: 0.5866 | Patience: 2/10
Epoch 30/50 | Loss: 0.6241 | Acc: 0.6477 | Patience: 0/10
Epoch 35/50 | Loss: 0.5983 | Acc: 0.6463 | Patience: 1/10
Epoch 40/50 | Loss: 0.5601 | Acc: 0.7202 | Patience: 1/10
Epoch 45/50 | Loss: 0.4614 | Acc: 0.7670 | Patience: 0/10
Epoch 50/50 | Loss: 0.4325 | Acc: 0.7727 | Patience: 3/10


In [31]:
# Restaurar mejor modelo
if best_model_state is not None:
    model.load_state_dict(best_model_state)
    print("✅ Mejor modelo restaurado")

✅ Mejor modelo restaurado


============================================================================
EVALUACIÓN EN TEST
============================================================================

In [32]:
print("\n" + "="*70)
print("EVALUACIÓN EN TEST")
print("="*70)


EVALUACIÓN EN TEST


In [33]:
model.eval()
all_preds = []
all_labels = []

In [34]:
with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X = batch_X.to(DEVICE)
        outputs = model(batch_X)
        _, predicted = torch.max(outputs, 1)
        
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(batch_y.numpy())

In [35]:
# Métricas
acc = accuracy_score(all_labels, all_preds)
prec = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
rec = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)
cm = confusion_matrix(all_labels, all_preds)

In [36]:
print(f"\n🎯 RESULTADOS:")
print(f"  Accuracy:  {acc:.4f} ({acc*100:.2f}%)")
print(f"  Precision: {prec:.4f}")
print(f"  Recall:    {rec:.4f}")
print(f"  F1-Score:  {f1:.4f}")
print(f"\nConfusion Matrix:")
print(f"  {cm}")
print(f"\n  Left correct:  {cm[0,0]}/{cm[0].sum()} = {cm[0,0]/cm[0].sum()*100:.1f}%")
print(f"  Right correct: {cm[1,1]}/{cm[1].sum()} = {cm[1,1]/cm[1].sum()*100:.1f}%")


🎯 RESULTADOS:
  Accuracy:  0.4716 (47.16%)
  Precision: 0.4707
  Recall:    0.4716
  F1-Score:  0.4677

Confusion Matrix:
  [[49 39]
 [54 34]]

  Left correct:  49/88 = 55.7%
  Right correct: 34/88 = 38.6%


============================================================================
GUARDAR RESULTADOS
============================================================================

In [37]:
# Guardar modelo
torch.save(model.state_dict(), OUT_DIR / 'deepconvnet_baseline.pth')

In [38]:
# Guardar métricas
metrics = {
    'accuracy': acc,
    'precision': prec,
    'recall': rec,
    'f1': f1,
    'confusion_matrix': cm,
    'history': history
}
np.save(OUT_DIR / 'metrics.npy', metrics)

In [39]:
# Guardar resumen
with open(OUT_DIR / 'summary.txt', 'w') as f:
    f.write("DEEPCONVNET BASELINE - RESULTADOS\n")
    f.write("="*70 + "\n\n")
    f.write(f"Dataset: {len(X_train)} train, {len(X_test)} test\n")
    f.write(f"Parámetros: {n_params:,}\n\n")
    f.write(f"Test Accuracy:  {acc:.4f} ({acc*100:.2f}%)\n")
    f.write(f"Test Precision: {prec:.4f}\n")
    f.write(f"Test Recall:    {rec:.4f}\n")
    f.write(f"Test F1-Score:  {f1:.4f}\n\n")
    f.write(f"Confusion Matrix:\n{cm}\n")

In [40]:
print(f"\n✅ Resultados guardados en: {OUT_DIR}")
print("="*70)
print("COMPLETADO")
print("="*70)


✅ Resultados guardados en: /Users/manueljurado/Downloads/datos_BCI/results/deepconvnet
COMPLETADO
